# EmbeddingGemma Embedding futtatás RunPod GPU-n

In [ ]:
%pip install faiss-cpu tqdm transformers accelerate huggingface_hub

/Users/zelenyianszkimate/Documents/CourtRankRL/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import json
from pathlib import Path
from typing import List
import math

import faiss
import numpy as np
import torch
from tqdm import tqdm
from transformers import AutoModel, AutoTokenizer

# HuggingFace bejelentkezés
from huggingface_hub import login
import os

# HF token betöltése .env fájlból
from dotenv import load_dotenv
load_dotenv()  # .env fájl betöltése

hf_token = os.getenv('HUGGINGFACE_TOKEN')
if hf_token:
    login(token=hf_token)
    print("✅ HuggingFace bejelentkezés sikeres")
else:
    print("⚠️  HF token nem található - egyes modellek korlátozottak lehetnek")


## Paraméterek
Állítsd be a bemeneti/ kimeneti elérési utakat.

In [ ]:
# Paraméterek beállítása - RunPod önálló futtatáshoz
# MINDEN elérési út és konfiguráció hardcoded a függetlenség érdekében

# Elérési utak (RunPod munkakönyvtárhoz igazítva)
chunks_path = Path("/workspace/data/processed/chunks.jsonl")
faiss_path = Path("/workspace/data/index/faiss_index.bin")
chunk_map_path = Path("/workspace/data/index/chunk_id_map.json")

# Modell konfiguráció
model_name = "google/embeddinggemma-300m"

# Memória optimalizált paraméterek
batch_size = 32  # GPU memória problémák elkerülésére
max_length = 512  # Csökkentett max token hossz

print("RunPod önálló notebook konfiguráció:")
print(f"Chunks path: {chunks_path}")
print(f"FAISS path: {faiss_path}")
print(f"Chunk map path: {chunk_map_path}")
print(f"Model: {model_name}")
print(f"Batch size: {batch_size}")
print(f"Max length: {max_length}")
print("Minden konfiguráció beállítva - notebook önállóan futtatható!")


NameError: name '__file__' is not defined

## Modell betöltése
Feltételezzük, hogy GPU elérhető (`torch.cuda.is_available()`).

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Default érték az embedding_dim-nek
embedding_dim = 768  # EmbeddingGemma-300m default dimenziója

# Memória optimalizálás
if device == 'cuda':
    torch.cuda.empty_cache()
    print(f"GPU memória előtte: {torch.cuda.memory_allocated()/1024**3:.1f} GB")

    # Modell betöltése memória optimalizált módon
    print(f"Modell betöltés: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

    try:
        # Próbáljuk meg az accelerate-t használni (ha telepítve van)
        model = AutoModel.from_pretrained(
            model_name,
            trust_remote_code=True,
            dtype=torch.float16,  # FP16 használata memória csökkentésére
            low_cpu_mem_usage=True,    # Alacsony CPU memória használat
            device_map="auto"          # Automatikus eszköz elhelyezés
        ).to(device)
        print("Accelerate használata sikeres")
    except (ImportError, ValueError) as e:
        # Fallback megoldás ha nincs accelerate
        print(f"Accelerate hiba, fallback megoldás: {e}")
        print("Modell betöltés CPU-ra, majd áthelyezés GPU-ra...")
        model = AutoModel.from_pretrained(
            model_name,
            trust_remote_code=True,
            dtype=torch.float16,
            low_cpu_mem_usage=True
        )
        model = model.to(device)
        print("Modell sikeresen betöltve GPU-ra")

    model.eval()
    try:
        embedding_dim = model.config.hidden_size
    except:
        pass  # Már van default érték beállítva

    if device == 'cuda':
        torch.cuda.empty_cache()
        print(f"GPU memória utána: {torch.cuda.memory_allocated()/1024**3:.1f} GB")

print(f"Model device: {device}")
print(f"Embedding dimension: {embedding_dim}")
print("Model loaded successfully!")
print(f"Batch size: {batch_size}, Max length: {max_length}")

## Embedding függvény
Egyszerű batch feldolgozás GPU-n.

In [ ]:
def embed_batch(texts: List[str]) -> np.ndarray:
    if not texts:
        return np.zeros((0, embedding_dim), dtype=np.float32)

    # Memória optimalizálás - kisebb batch-ek kezelése
    if len(texts) > batch_size:
        # Ha túl nagy a batch, feldaraboljuk
        all_embeddings = []
        for i in range(0, len(texts), batch_size):
            sub_texts = texts[i:i + batch_size]
            sub_embeddings = embed_batch(sub_texts)
            all_embeddings.append(sub_embeddings)
        return np.vstack(all_embeddings)

    inputs = tokenizer(
        texts,
        return_tensors='pt',
        truncation=True,
        max_length=max_length,
        padding=True
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        if hasattr(outputs, 'last_hidden_state'):
            embeddings = outputs.last_hidden_state[:, 0, :].float()
        else:
            embeddings = outputs.pooler_output.float()

    # Memória felszabadítás
    del inputs, outputs
    if device == 'cuda':
        torch.cuda.empty_cache()

    embeddings = embeddings.cpu().numpy()
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    norms[norms == 0] = 1.0
    return embeddings / norms


## FAISS index létrehozása
Az index paramétereit egyszerűre vesszük: nlist = sqrt(N), PQ m=64 (igazítva a dimenzióhoz).

In [ ]:
chunk_ids = []
index = None
embedding_dim = None
vector_count = 0

with chunks_path.open('r', encoding='utf-8') as handle:
    iterator = tqdm(handle, desc='Streaming embedding generálás')
    batch_texts = []
    batch_ids = []
    for line in iterator:
        line = line.strip()
        if not line:
            continue
        try:
            chunk = json.loads(line)
        except json.JSONDecodeError:
            continue
        chunk_id = str(chunk.get('chunk_id', '')).strip()
        text = chunk.get('text', '')
        if not isinstance(text, str):
            text = str(text)
        if not chunk_id or not text.strip():
            continue
        batch_ids.append(chunk_id)
        batch_texts.append(text)
        if len(batch_texts) >= batch_size:
            vectors = embed_batch(batch_texts)
            if vectors.size > 0:
                if index is None:
                    embedding_dim = vectors.shape[1]
                    index = faiss.IndexFlatIP(embedding_dim)
                index.add(vectors.astype(np.float32, copy=False))
                chunk_ids.extend(batch_ids)
                vector_count += vectors.shape[0]
            batch_texts.clear()
            batch_ids.clear()
    # Maradék batch kezelése
    if batch_texts:
        vectors = embed_batch(batch_texts)
        if vectors.size > 0:
            if index is None:
                embedding_dim = vectors.shape[1]
                index = faiss.IndexFlatIP(embedding_dim)
            index.add(vectors.astype(np.float32, copy=False))
            chunk_ids.extend(batch_ids)
            vector_count += vectors.shape[0]

# Ellenőrzés
if index is None or index.ntotal == 0:
    raise ValueError("Hiba: Nem sikerült embedding indexet létrehozni!")
print(f"Feldolgozott chunk-ok száma: {len(chunk_ids)}")
print(f"Vektorok száma az indexben: {index.ntotal}")
vector_count

## FAISS paraméterek és index tréning

In [ ]:
# FAISS paraméterek és index létrehozása
if index is None:
    raise ValueError("Hiba: Az index nem lett inicializálva!")
    
nlist = max(1, int(math.sqrt(vector_count)))
pq_m = 64
if embedding_dim % pq_m != 0:
    for candidate in range(pq_m, 0, -1):
        if embedding_dim % candidate == 0:
            pq_m = candidate
            break
pq_bits = 8

# Vektorok kinyerése az IndexFlatIP-ből
print("📊 Vektorok kinyerése az IndexFlatIP-ből...")
try:
    all_vectors = index.reconstruct_n(0, index.ntotal)
except AttributeError as err:
    raise AttributeError("Ez a FAISS build nem támogatja az 'xb' attribútumot; a reconstruct_n API-t kell használni a vektorok kinyeréséhez.") from err
if not isinstance(all_vectors, np.ndarray):
    vector_to_array = getattr(faiss, "vector_float_to_array", getattr(faiss, "vector_to_array", None))
    if vector_to_array is None:
        raise RuntimeError("Nem található FAISS segédfüggvény a vektorok numpy tömbbé alakításához.")
    all_vectors = vector_to_array(all_vectors)
    all_vectors = all_vectors.reshape(index.ntotal, embedding_dim)
all_vectors = np.ascontiguousarray(all_vectors, dtype=np.float32)
print(f"✅ Vektorok kinyerve: {all_vectors.shape}")

# Kvantált index létrehozása
quantizer = faiss.IndexFlatIP(embedding_dim)
new_index = faiss.IndexIVFPQ(quantizer, embedding_dim, nlist, pq_m, pq_bits, faiss.METRIC_INNER_PRODUCT)

# Index tréning és feltöltés
new_index.train(all_vectors)
new_index.add(all_vectors)

# Csere
index = new_index

# Ellenőrzés
print("Index created with", len(chunk_ids), "vectors")



## Eredmények mentése

In [ ]:
faiss_path.parent.mkdir(parents=True, exist_ok=True)
faiss.write_index(index, str(faiss_path))
chunk_id_map = {str(i): cid for i, cid in enumerate(chunk_ids)}
with chunk_map_path.open('w', encoding='utf-8') as f:
    json.dump(chunk_id_map, f)
faiss_path, chunk_map_path